###Online LDA for Wikipedia
Allison Hwang and Albert Kuo

In [2]:
import requests
import json 
import re
import itertools

A function to parse the test documents that we were given:

In [3]:
def parse_doc(a):
    a = re.search(r'<text.*?>(.*)</text', a, flags=re.DOTALL).group(1)
    a = re.sub(r'\n', ' ', a)
    a = re.sub(r'\{\{.*?\}\}', r'', a)
    a = re.sub(r'\[\[Category:.*', '', a)
    a = re.sub(r'==\s*[Ss]ource\s*==.*', '', a)
    a = re.sub(r'==\s*[Rr]eferences\s*==.*', '', a)
    a = re.sub(r'==\s*[Ee]xternal [Ll]inks\s*==.*', '', a)
    a = re.sub(r'==\s*[Ee]xternal [Ll]inks and [Rr]eferences==\s*', '', a)
    a = re.sub(r'==\s*[Ss]ee [Aa]lso\s*==.*', '', a)
    a = re.sub(r'http://[^\s]*', '', a)
    a = re.sub(r'\[\[Image:.*?\]\]', '', a)
    a = re.sub(r'Image:.*?\|', '', a)
    a = re.sub(r'\[\[.*?\|*([^\|]*?)\]\]', r'\1', a)
    a = re.sub(r'\&lt;.*?&gt;', '', a)
    return a

In [4]:
articles = []
for item in requests.get("http://s3.amazonaws.com/stat-37601/wiki.json", stream=True).iter_lines():
    j = json.loads(item)
    articles.append(parse_doc(j['body']))

function that calculates similarity between two documents based on Hellinger distance.

In [5]:
import math
def doc_similarity(doc1, doc2):
    length = min(len(doc1), len(doc2))
    sim = 0
    for i in range(length):
        sim += math.pow((doc1[i]-doc2[i]), 2)
    return sim

I made changes to onlinewikipedia.py to get our desired output. First I run on 200 iterations to train the model. By specifically using the do_e_step function from onlineldavb, I'm able to get the gammas for the test document set based on the lambdas from the model that we fitted. Then I normalize the gammas before calculating Hellinger distances to find similar documents.

In [8]:
import cPickle, string, numpy, getopt, sys, random, time, re, pprint

import onlineldavb
import wikirandom

def main():
    """
    Downloads and analyzes a bunch of random Wikipedia articles using
    online VB for LDA.
    """

    # The number of documents to analyze each iteration
    batchsize = 64
    # The total number of documents in Wikipedia
    D = 3.3e6
    # The number of topics
    K = 100
    
    # 200 iterations
    documentstoanalyze = 200

    # Our vocabulary
    vocab = file('./dictnostops.txt').readlines()
    W = len(vocab)

    # Initialize the algorithm with alpha=1/K, eta=1/K, tau_0=1024, kappa=0.7
    olda = onlineldavb.OnlineLDA(vocab, K, D, 1./K, 1./K, 1024., 0.7)
    # Run until we've seen D documents. (Feel free to interrupt *much*
    # sooner than this.)
    for iteration in range(0, documentstoanalyze):
        # Download some articles
        (docset, articlenames) = \
            wikirandom.get_random_wikipedia_articles(batchsize)


        # Give them to online LDA
        (gamma, bound) = olda.update_lambda(docset)
        # Compute an estimate of held-out perplexity
        (wordids, wordcts) = onlineldavb.parse_doc_list(docset, olda._vocab)
        perwordbound = bound * len(docset) / (D * sum(map(sum, wordcts)))
        print '%d:  rho_t = %f,  held-out perplexity estimate = %f' % \
            (iteration, olda._rhot, numpy.exp(-perwordbound))

    # get gammas for test documents
    test_docs = articles
    gammas = olda.do_e_step(test_docs)[0] 

    # normalize
    for i in range(len(gammas)):
        article = gammas[i]
        total = sum(article)
        for j in range(len(article)):
            gammas[i][j] = gammas[i][j] / total
    
    # iterate through every pair of documents, find similarity
    sims = []
    n = range(len(gammas))
    for combo in itertools.combinations(n, 2):
        sim = doc_similarity(gammas[combo[0]],gammas[combo[1]])
        s = (combo, sim)
        sims.append(s)
    # print out the 20 pairs of most similar documents
    sims.sort(key=lambda x: x[1], reverse=True)
    for i in range(20):
        print sims[i]
                    
if __name__ == '__main__':
    main()


downloaded 0/64 articles...
downloaded Abraham_Patras. parsing...
downloaded Germ_theory_of_disease. parsing...
downloaded Ettore_Marchiafava. parsing...
downloaded Mariano_Raffo. parsing...
downloaded Australian_War_Memorial. parsing...
downloaded John_Komnenos_the_Fat. parsing...
downloaded Ombrone. parsing...
downloaded Shushanik_Kurghinian. parsing...
downloaded 8/64 articles...
downloaded Tourism_in_the_Gambia. parsing...
downloaded Elsa_Chauvel. parsing...
downloaded Hilbert%E2%80%93Speiser_theorem. parsing...
downloaded Hugh_le_Despencer,_1st_Baron_le_Despencer. parsing...
downloaded Richard_Hastings,_Baron_Welles. parsing...
downloaded Kim_Young-ok. parsing...
downloaded Suprameatal_triangle. parsing...
downloaded Bulgaria_in_the_Eurovision_Song_Contest_2007. parsing...
downloaded 16/64 articles...
downloaded Gasteranthus_extinctus. parsing...
downloaded Velutina_schneideri. parsing...
downloaded Joel_Furr. parsing...
downloaded Typhoon_Dujuan_(2003). parsing...
downloaded Aidu

Exception in thread Thread-25697:
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "wikirandom.py", line 88, in run
    (article, articlename) = get_random_wikipedia_article()
  File "wikirandom.py", line 49, in get_random_wikipedia_article
    f = urllib2.urlopen(req)
  File "//anaconda/lib/python2.7/urllib2.py", line 154, in urlopen
    return opener.open(url, data, timeout)
  File "//anaconda/lib/python2.7/urllib2.py", line 431, in open
    response = self._open(req, data)
  File "//anaconda/lib/python2.7/urllib2.py", line 449, in _open
    '_open', req)
  File "//anaconda/lib/python2.7/urllib2.py", line 409, in _call_chain
    result = func(*args)
  File "//anaconda/lib/python2.7/urllib2.py", line 1227, in http_open
    return self.do_open(httplib.HTTPConnection, req)
  File "//anaconda/lib/python2.7/urllib2.py", line 1200, in do_open
    r = h.getresponse(buffering=True)
  File "//anaconda/lib/p

I measured similarity based on the Hellinger distance.<br>
Here are some of the top pairs based on similarity (with their similarity scores):<br>
((7, 48), 0.26217822831890164)<br>
((7, 55), 0.25585975068505057)<br>
((7, 16), 0.2366617492527577)<br>
((7, 15), 0.23408082822721393)<br>
((7, 19), 0.2330429599600336)<br>
((5, 48), 0.22515447581424294)<br>
((7, 12), 0.2243611692469096)<br>
((5, 55), 0.22184605902186455)<br>
((7, 13), 0.22163041640443001)<br>
((11, 48), 0.22048857138004507)<br>
((7, 37), 0.21770718194423044)<br>
((11, 55), 0.21398426687218033)<br>
((7, 52), 0.2138582084511958)<br>
((7, 17), 0.21304343567533454)<br>
((3, 48), 0.21220245884810757)<br>
((7, 14), 0.21156095612403497)<br>
((6, 48), 0.21155184934131024)<br>
((6, 55), 0.21063517962477618)<br>
((7, 42), 0.21034499798253642)<br>
((4, 48), 0.20866376017582597)<br>

Let's look at this pair, (7, 48). Article 7 is about a competitive swimmer and article 48 is about a Japanese bank. One common topic may be that certain Japanese locations are mentioned in article 7 as places where the swimmer had competed. Also, the consistent mentioning of "gold" and "silver" medals may seem like a finance topic, which the Japanese bank article likely has as well.

In [7]:
articles[7]

u" | strokes        = Backstroke | club           = Longhorn Aquatics | collegeteam    = University of Texas | birth_date     =  | birth_place    = Irvine, California | death_date     = | death_place    = | height         =  | weight         =  | medaltemplates =   }}                                           }}  '''Aaron Wells Peirsol''' (born July 23, 1983) is a former American competition  swimmer who specialized in the backstroke.  He is a three-time Olympian and seven-time Olympic medalist (five gold, two silver).  As a member of the U.S. national team, he holds the world record in the men's 4\xd7100-meter medley relay (long course).  Individually, he currently holds the world record in the 100-meter and 200-meter backstroke events (long course).  In February 2011, Peirsol announced his retirement, saying, &quot;I ended up doing everything I set out to do.&quot;   Peirsol's successes have earned him the American Swimmer of the Year Award once.  He has won a total of thirty-six med

In [8]:
articles[48]

u"  , Tokyo]] '''Japan Post Bank Company Limited''' (\u682a\u5f0f\u4f1a\u793e\u3086\u3046\u3061\u3087\u9280\u884c Kabushiki-gaisha Y\u016b-cho Gink\u014d, commonly abbreviated to \u3086\u3046\u3061\u3087\u9280\u884c (Y\u016b-cho Gink\u014d), or just \u3086\u3046\u3061\u3087(Y\u016b-cho)), is a Japanese bank headquartered in Tokyo which is part of the Japan Post Holdings postal and financial services group.  As of November 2008 it was reported as being the world's biggest deposit holder. It is one of only two banks to have branches in every prefecture in Japan, the other being Mizuho Bank.  ==History==  Postal savings was introduced to Japan in 1875 and operated as a government department until privatization of the postal service was passed under the government of Prime Minister Junichiro Koizumi.  The bank was established on 1 September 2006, as part of the reorganisation of Japan Post into Japan Post Holdings.  Prior to 2009, Japan Post was not connected to the Japanese Bankers Associ

Now, looking at the second pairing, we have article 55, which is about Japanese company Mitsubishi Tokyo Financial Group, which would have similarities to article 7 for the same reasons as above. I am noticing that a lot of these top pairings include article 7, which may be due to the longer length of article 7 than some of these other articles. Thus it is likely to have more topics than average, and therefore be more similar

In [9]:
articles[55]

u"  |genre            = |fate             = |predecessor      = Mitsubishi Tokyo Financial Group, Inc.UFJ Holdings, Inc. |successor        = |foundation       = October 1, 2005(by merger) |founder          = |defunct          = |location_city    =  Chiyoda-ku, Tokyo |location_country =  Japan |location         = |locations        = |area_served      = Worldwide |key_people       = Takamune Okihara (Chairman) Tatsuo Wakabayashi (Deputy Chairman) Nobuyuki Hirano (President &amp; CEO) |industry         = Banking, Financial services |products         = |production       = |services         = Personal BankingCorporate BankingInvestment BankingInvestment ManagementWealth ManagementMortgageCredit Cards |revenue          =   (2013) |net_income       =   (2013) |aum              = |assets           =   (2013) |equity           =   (2013) |owner            = |num_employees    = 80,900 (2013) |parent           = Mitsubishi Group |divisions        = |subsid           = The Bank of Tokyo-Mitsubishi

The pairings included article pairs with some similarities but did not pick out every pair that, upon my visual inspection, are very topically related.<br>
For instance (12, 13) was not in the top 20 pairs. <br>
Articles 12 and 13 are both related to computer languages.

In [15]:
articles[12]

u" | designer               = Guido van Rossum | developer              = Python Software Foundation | latest_release_version = 3.4.0 /2.7.6 / | latest_preview_version = 3.3.5 rc1 /3.4.0 rc2 / | typing                 = duck, dynamic, strong | implementations        = CPython, PyPy, IronPython, Jython | dialects               = Cython, RPython, Stackless Python | influenced_by          = ABC, ALGOL 68, C, C++, Dylan, Haskell, Icon, Java, Lisp, Modula-3, Perl | influenced             = Boo, Cobra, D, F#, Falcon, Go, Groovy, JavaScript, Ruby | operating_system       = Cross-platform | license                = Python Software Foundation License | website                =  | file_ext               = .py, .pyw, .pyc, .pyo, .pyd | wikibooks              = Python Programming }}  '''Python''' is a widely used general-purpose, high-level programming language. Its design philosophy emphasizes code readability, and its syntax allows programmers to express concepts in fewer lines of code than woul

In [18]:
articles[13]

u"    | designer = James Gosling andSun Microsystems | developer = Oracle Corporation | written_in = c++ | latest release version = Java Standard Edition 8 Update 5 (1.8.0_5) | latest release date    =  | latest preview version =  | latest preview date    =  | frequently updated     =  | turing-complete = Yes | typing = Static, strong, safe, nominative, manifest | implementations = OpenJDK, many others | influenced_by = Ada 83, C++, C#,Java 5.0 added several new language features (the enhanced for loop, autoboxing, varargs and annotations), after they were introduced in the similar (and competing) C# language [ [ Eiffel, Generic Java, Mesa, Modula-3, Oberon,Niklaus Wirth stated on a number of public occasions, e.g. in a lecture at the Polytechnic Museum, Moscow in September, 2005 (several independent first-hand accounts in Russian exist, e.g. one with an audio recording: ), that the Sun Java design team licenced the Oberon compiler sources a number of years prior to the release of Java